In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('cluster2').getOrCreate()

In [5]:
dataset = spark.read.csv('hdfs://nn-homedepot.s3s.altiscale.com:8020/user/mlakshmikanthaiah/personl_test_files/seeds_dataset.csv',inferSchema=True,header=True)

In [7]:
dataset.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



In [9]:
from pyspark.ml.clustering import KMeans

In [13]:
columns = dataset.columns
columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [14]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = dataset.columns,outputCol='features')

In [17]:
final_data = assembler.transform(dataset)
final_dataset = final_data.select('features')

In [19]:
from pyspark.ml.feature import StandardScaler
scalar = StandardScaler(inputCol ='features',outputCol='scaledFeatures')

scalar_model = scalar.fit(final_dataset)
final_dataset = scalar_model.transform(final_dataset)

In [20]:
final_dataset.head(1)

[Row(features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [21]:
kmeans = KMeans(featuresCol='scaledFeatures',k=3)
model = kmeans.fit(final_dataset)

In [22]:
wssse = model.computeCost(final_dataset)
print('wssse',wssse)

('wssse', 429.07559671506715)


In [23]:
centers = model.clusterCenters()
print(centers)

[array([  6.31670546,  12.37109759,  37.39491396,  13.91155062,
         9.748067  ,   2.39849968,  12.2661748 ]), array([  4.87257659,  10.88120146,  37.27692543,  12.3410157 ,
         8.55443412,   1.81649011,  10.32998598]), array([  4.06105916,  10.13979506,  35.80536984,  11.82133095,
         7.50395937,   3.27184732,  10.42126018])]


In [25]:
model.transform(final_dataset).select('prediction').show()

+----------+
|prediction|
+----------+
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         0|
|         0|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         2|
+----------+
only showing top 20 rows

